1) transfer.csv pre-processing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data_transfers=pd.read_csv('transfers.csv')

data_transfers.head()


,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
0,195778,2026-06-30,25/26,79,27,VfB Stuttgart,Bayern Munich,0.0,12000000.0,Alexander Nübel
1,569033,2026-06-30,25/26,39,27,1.FSV Mainz 05,Bayern Munich,0.0,4000000.0,Armindo Sieb
2,626913,2026-06-30,25/26,398,380,Lazio,Salernitana,0.0,15000000.0,Boulaye Dia
3,1047109,2026-06-30,25/26,2672,265,APO Levadiakos,Panathinaikos,0.0,100000.0,Georgios Katris
4,360791,2025-12-31,25/26,6418,1030,Panetolikos,Argentinos Jrs.,0.0,1200000.0,Lucas Chaves


In [ ]:
# Find the transfer(s) with the transfer date '2024-06-30'
transfer_in_2024 = data_transfers[data_transfers['transfer_date'] == '2024-06-30']

# Display the relevant information about these transfers
transfer_in_2024[['player_name', 'to_club_name', 'from_club_name', 'transfer_fee', 'market_value_in_eur']]

,player_name,to_club_name,from_club_name,transfer_fee,market_value_in_eur
4897,Alessio Cragno,Monza,Sassuolo,0.0,1500000.0
4898,Massimo Coda,Genoa,Cremonese,0.0,1000000.0
4899,Marko Arnautovic,Bologna,Inter,0.0,4000000.0
4900,Ivan Perisic,Tottenham,Hajduk Split,0.0,2000000.0
4901,Anders Jacobsen,AC Horsens,Vejle BK,0.0,300000.0
...,...,...,...,...,...
6137,Matar Manga,Moreirense,Real SC,0.0,100000.0
6138,Ebrima Ndow,Moreirense,Mirandela,0.0,100000.0
6139,Nikita Lednev,FK Minaj U19,FK Khust,0.0,NaN
6140,Gökdeniz Kara,Pendik U19,Afyonspor,0.0,25000.0


2) transfer data analysis
   -  identifing the market values of each transfer per season

In [ ]:
import plotly.express as px

data_transfers_filtered = data_transfers[
    (data_transfers['transfer_season'] != '25/26') &
    (data_transfers['transfer_season'] != '04/05') &
    (data_transfers['transfer_season'] != '02/03') &
    (data_transfers['transfer_fee'] > 0)
]

# Convert 'transfer_season' to a string type and sort by season in ascending order
data_transfers_filtered['transfer_season'] = data_transfers_filtered['transfer_season'].astype(str)
data_transfers_filtered = data_transfers_filtered.sort_values('transfer_season', ascending=True)

# Recreate the plot with sorted and filtered data
fig_transfer = px.box(data_transfers_filtered,
                      x='transfer_season',
                      y='transfer_fee',
                      title='Market Value Distribution by Transfer Season',
                      points="outliers",  # Show only the outliers
                      color='transfer_season',  # Color by transfer season
                      hover_data=['player_name', 'market_value_in_eur'])  # Add hover data for player names

# Adjust the layout for better readability
fig_transfer.update_layout(xaxis_title='Transfer Season', yaxis_title='Market Value in EUR', showlegend=False)
fig_transfer.update_xaxes(tickangle=-45)  # Rotate x-axis labels for better readability

# Show the plot
fig_transfer.show()

<ipython-input-3-5b348cd9f6b6>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_transfers_filtered['transfer_season'] = data_transfers_filtered['transfer_season'].astype(str)


 - Computing the average transfer fee per season

In [ ]:
# Remove rows with zero or null transfer fees
data_transfers_filtered = data_transfers_filtered[data_transfers_filtered['transfer_fee'] > 0]

# Calculate the average transfer fee per season excluding zero values
average_transfer_fee_per_season = data_transfers_filtered.groupby('transfer_season')['transfer_fee'].mean().reset_index()

# Format the transfer_fee column to show values with thousand separators
average_transfer_fee_per_season['transfer_fee'] = average_transfer_fee_per_season['transfer_fee'].apply(lambda x: f"{x:,.0f}")

# Convert the 'transfer_fee' column back to numeric for plotting
average_transfer_fee_per_season['transfer_fee'] = average_transfer_fee_per_season['transfer_fee'].str.replace(',', '').astype(float)

# Create a bar chart with Plotly Express
fig = px.bar(
    average_transfer_fee_per_season,
    x='transfer_season',
    y='transfer_fee',
    color='transfer_fee',
    title='Average Transfer Fee Per Season',
    labels={'transfer_season': 'Transfer Season', 'transfer_fee': 'Average Transfer Fee (in EUR)'},
    color_continuous_scale='Blues'  # Color the bars based on their values
)

# Customize the layout
fig.update_layout(
    xaxis_title='Transfer Season',
    yaxis_title='Average Transfer Fee (in EUR)',
    font=dict(size=12),
)

# Show the plot
fig.show()


 - data analysis to study the correlation between transfer_fee and market_value
    
    . identifyng the top overpaid players


In [ ]:
# Filter out rows where either transfer_fee or market_value_in_eur is zero or null
data_transfers_filtered_non_zero = data_transfers_filtered[(data_transfers_filtered['transfer_fee'] > 0) & (data_transfers_filtered['market_value_in_eur'] > 0)]

# Calculate the difference between transfer_fee and market_value_in_eur
data_transfers_filtered_non_zero['value_diff'] = data_transfers_filtered_non_zero['transfer_fee'] - data_transfers_filtered_non_zero['market_value_in_eur']

# Find the top 10 most "Overpaid Players" where transfer_fee is higher than market_value_in_eur
overpaid_players = data_transfers_filtered_non_zero[data_transfers_filtered_non_zero['value_diff'] > 0].sort_values(by='value_diff', ascending=False)

# Find the top 10 most "Underpaid Players" where transfer_fee is lower than market_value_in_eur
underpaid_players = data_transfers_filtered_non_zero[data_transfers_filtered_non_zero['value_diff'] < 0].sort_values(by='value_diff', ascending=True)

# Get the top 10 overpaid and underpaid players with their transfer season
top_10_overpaid_players = overpaid_players[['player_name', 'transfer_season', 'transfer_fee', 'market_value_in_eur', 'value_diff']].head(10)
top_10_underpaid_players = underpaid_players[['player_name', 'transfer_season', 'transfer_fee', 'market_value_in_eur', 'value_diff']].head(10)

# Format the transfer_fee, market_value_in_eur, and value_diff with thousand separators for better readability
top_10_overpaid_players['transfer_fee'] = top_10_overpaid_players['transfer_fee'].apply(lambda x: f"{x:,.0f}")
top_10_overpaid_players['market_value_in_eur'] = top_10_overpaid_players['market_value_in_eur'].apply(lambda x: f"{x:,.0f}")
top_10_overpaid_players['value_diff'] = top_10_overpaid_players['value_diff'].apply(lambda x: f"{x:,.0f}")

top_10_underpaid_players['transfer_fee'] = top_10_underpaid_players['transfer_fee'].apply(lambda x: f"{x:,.0f}")
top_10_underpaid_players['market_value_in_eur'] = top_10_underpaid_players['market_value_in_eur'].apply(lambda x: f"{x:,.0f}")
top_10_underpaid_players['value_diff'] = top_10_underpaid_players['value_diff'].apply(lambda x: f"{x:,.0f}")

<ipython-input-5-e61ad0d266de>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



- identify the top underpaid players

In [ ]:
print("Top 10 Overrated Players")
top_10_overpaid_players


Top 10 Overrated Players


,player_name,transfer_season,transfer_fee,market_value_in_eur,value_diff
50124,Ousmane Dembélé,17/18,"135,000,000","33,000,000","102,000,000"
15132,Enzo Fernández,22/23,"121,000,000","55,000,000","66,000,000"
17139,Antony,22/23,"95,000,000","35,000,000","60,000,000"
44828,Kepa Arrizabalaga,18/19,"80,000,000","20,000,000","60,000,000"
46219,Kylian Mbappé,18/19,"180,000,000","120,000,000","60,000,000"
39619,João Félix,19/20,"127,200,000","70,000,000","57,200,000"
49346,Virgil van Dijk,17/18,"84,650,000","30,000,000","54,650,000"
25448,Jack Grealish,21/22,"117,500,000","65,000,000","52,500,000"
59121,Anthony Martial,15/16,"60,000,000","8,000,000","52,000,000"
49229,Philippe Coutinho,17/18,"135,000,000","90,000,000","45,000,000"


In [ ]:
# Display the top 10 underpaid players
print("\nTop 10 Underpaid Players")
top_10_underpaid_players


Top 10 Underpaid Players


,player_name,transfer_season,transfer_fee,market_value_in_eur,value_diff
19573,Erling Haaland,22/23,"60,000,000","150,000,000","-90,000,000"
36874,Christian Eriksen,19/20,"27,000,000","90,000,000","-63,000,000"
39074,Nabil Fekir,19/20,"19,750,000","60,000,000","-40,250,000"
39954,Luka Jović,19/20,"22,340,000","60,000,000","-37,660,000"
49067,Alexis Sánchez,17/18,"34,000,000","70,000,000","-36,000,000"
10945,Xavi Simons,23/24,"4,000,000","40,000,000","-36,000,000"
37077,Duván Zapata,19/20,"12,000,000","45,000,000","-33,000,000"
17117,Fabián Ruiz,22/23,"22,500,000","55,000,000","-32,500,000"
16878,Carlos Soler,22/23,"18,000,000","50,000,000","-32,000,000"
34156,Leroy Sané,20/21,"49,000,000","80,000,000","-31,000,000"


this exemples show that transfer_fee and market_values could not be correlated and other factors help create this differnce

3) Applying FCA to study the correspandce between transfer_fee and market_value_in_eur

In [ ]:
!pip install prince

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.5/232.5 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [ ]:
import pandas as pd
import numpy as np
import prince  # Library for Correspondence Analysis
import matplotlib.pyplot as plt

# Simulated dataset
data_transfers = pd.DataFrame({
    "transfer_fee": [500000, 2000000, 15000000, 70000000, np.nan, 30000000, 5000000],
    "market_value_in_eur": [700000, 1800000, 20000000, 65000000, 10000000, np.nan, 4500000]
})

# Drop missing values
data_transfers = data_transfers.dropna()

# Discretize transfer_fee into bins
bins_fee = [0, 1000000, 10000000, 50000000, np.inf]
labels_fee = ["low_fee", "medium_fee", "high_fee", "very_high_fee"]
data_transfers["fee_range"] = pd.cut(data_transfers["transfer_fee"], bins=bins_fee, labels=labels_fee)

# Discretize market_value_in_eur into bins
bins_value = [0, 1000000, 10000000, 50000000, np.inf]
labels_value = ["low_value", "medium_value", "high_value", "very_high_value"]
data_transfers["value_range"] = pd.cut(data_transfers["market_value_in_eur"], bins=bins_value, labels=labels_value)

# Create a contingency table
contingency_table = pd.crosstab(data_transfers["fee_range"], data_transfers["value_range"])

# Perform Correspondence Analysis
ca = prince.CA(n_components=2)
ca_result = ca.fit(contingency_table)

# Extract coordinates for rows and columns
row_coords = ca.row_coordinates(contingency_table)
col_coords = ca.column_coordinates(contingency_table)
# Retrieve explained inertia (variance proportions)
explained_inertia = ca.eigenvalues_ / ca.eigenvalues_.sum()



# Print explained inertia for interpretation
print("Explained Variance (Inertia) by Dimension:")
for i, inertia in enumerate(explained_inertia, 1):
    print(f"  Dimension {i}: {inertia*100:.2f}%")

# Conclusion
if sum(explained_inertia[:2]) < 10:  # Example threshold for weak correlation
    print("The correlation between transfer fees and market values is weak.")

Explained Variance (Inertia) by Dimension:
  Dimension 1: 50.00%
  Dimension 2: 50.00%
The correlation between transfer fees and market values is weak.


If only 10% of the variance is explained by these two dimensions, this suggests that there are many other factors influencing the transfer fee, which market value alone cannot account for.

In [ ]:
import prince
import numpy as np




data = data_transfers.dropna(subset=["transfer_fee", "market_value_in_eur"])

# Discretize transfer_fee into bins
bins_fee = [0, 1000000, 10000000, 50000000, np.inf]
labels_fee = ["low_fee", "medium_fee", "high_fee", "very_high_fee"]
data["fee_range"] = pd.cut(data["transfer_fee"], bins=bins_fee, labels=labels_fee)

# Discretize market_value_in_eur into bins
bins_value = [0, 1000000, 10000000, 50000000, np.inf]
labels_value = ["low_value", "medium_value", "high_value", "very_high_value"]
data["value_range"] = pd.cut(data["market_value_in_eur"], bins=bins_value, labels=labels_value)

# Construct a contingency table
contingency_table = pd.crosstab(data["fee_range"], data["value_range"])

# Perform Correspondence Analysis
ca = prince.CA(n_components=2)
ca_result = ca.fit(contingency_table)

# Transform data for visualization
rows, cols = ca.row_coordinates(contingency_table), ca.column_coordinates(contingency_table)

# Display results
print("Row Coordinates:")
print(rows)
print("\nColumn Coordinates:")
print(cols)



Row Coordinates:
                 0         1
fee_range                   
low_fee        2.0  0.000000
medium_fee    -0.5 -0.462646
high_fee      -0.5  1.902062
very_high_fee -0.5 -0.976770

Column Coordinates:
                   0             1
value_range                       
low_value        2.0  3.072780e-17
medium_value    -0.5 -4.626458e-01
high_value      -0.5  1.902062e+00
very_high_value -0.5 -9.767700e-01


this show that the transfers operations can't be explain by the market values . So we need to study the other factors that identify the process of players_transfers.
